In [12]:
API_KEY = 'INPUT API KEY HERE'

import boto3
import json
import io
from io import BytesIO
from io import StringIO
import pandas as pd
import os
import requests
import json

In [2]:
class EApy(object):
    def __init__(self,beginning,ending,API_KEY):
        self.API_KEY = API_KEY
        self.beginning = beginning
        self.ending = ending
        self.get_license()
        self.get_data()
    
    def get_license(self):
        url = 'https://api.energyaspects.com/data/datasets/timeseries'
        df_license = pd.DataFrame()
        params = {'api_key': API_KEY}
        response = requests.get(url, params=params)
        json_data = json.loads(response.text)
        for i in range(len(json_data)):
            metadata = json_data[i]['metadata']
            df = pd.DataFrame(metadata, index=[i])
            df['id'] = json_data[i]['dataset_id']
            df_license = pd.concat([df_license,df])
        df_license = df_license.drop_duplicates(subset='id', keep='first')
        self.license = df_license
        
    def get_data(self):
        df_license = self.license
        df_license_yes = df_license[df_license['licensed'] == 'yes']
        ids = df_license_yes['id'].values.tolist()
        l = ids
        n = 200 #USE 200 IN NORMAL CASE AS IT IS THE MAX NUMBER OF DATASETS THAT CAN BE RETRIEVED
        ids_chunk = [l[i:i + n] for i in range(0, len(l), n)]
        final_data = pd.DataFrame()
        url = 'https://api.energyaspects.com/data/timeseries'
        for z in ids_chunk:
            params = {'api_key': API_KEY, 'date_from': self.beginning,
                        'dataset_id': z}  # FIX THIS LATER put value as Z
            response = requests.get(url, params=params)
            
            json_data = json.loads(response.text)

            df_chunk = pd.DataFrame()

            if response.status_code == 200:
                json_data = json.loads(response.text)

                for i in range(len(json_data)):
                    values = json_data[i]['data']
                    metadata = json_data[i]['metadata']
                    metadata = pd.DataFrame(metadata, index=[0]).transpose().reset_index()
                    metadata = metadata.rename(columns={'index': 'header', 0: 'values'})
                    df = pd.DataFrame(values, index=['values']).transpose()

                    df['id'] = json_data[i]['dataset_id']

                    for j in metadata.index:
                        df['new'] = metadata['values'][j]
                        df = df.rename(columns=({'new': metadata['header'][j]}))

                    df_chunk = pd.concat([df_chunk, df], ignore_index=False)

            final_data = pd.concat([final_data, df_chunk], ignore_index=False)

        final_data = final_data[final_data.index <= self.ending]
        
        indexvalues = metadata['header'].values.tolist()
        indexvalues.append('id')

        indexvalues += [indexvalues.pop(4)]
        pivot_data = final_data.copy()
        pivot_data = pivot_data.pivot(columns=indexvalues)['values']
        self.pivot_data = pivot_data        

        final_data = final_data.reset_index()
        final_data = final_data.rename(columns={'index': 'delivery_date'})
        self.df = final_data
        
    def create_pivot_monthly(self,freq):
        df = self.pivot_data.copy()
        df = df.transpose().reset_index().copy()
        df = df[df['frequency'] == freq].copy()
        df.dropna(axis='columns', how='all', inplace=True)
        df['id'] = df['id'].astype(int)
        df.sort_values(by=['id'], inplace=True)
        return df
    
    def create_xlsx_model_file(self):
        df = self.df.copy()
        df = df[df['frequency']=='monthly'].copy()
        df_meta = df.drop(columns=['delivery_date','values']).drop_duplicates()
        cols = df_meta.columns
        cols = cols.insert(0,'delivery_date')
        df = df[['id','delivery_date','values']]
                
        return df, df_meta

In [13]:
'''RUN THE CLASS WHICH STORES THE MASTER DATA as variable "EA.df" '''
EA = EApy('2015-01-01','2024-12-31',API_KEY)

In [16]:
'''TO LOCAL COMPUTER: CREATE TABULAR MASTER FILE WITH ALL METADATA'''
fd = EA.df
fd.to_csv('Master_EA_Tabular.csv',index=False)

In [21]:
'''TO LOCAL COMPUTER: CREATE MASTER FILE IN PIVOT FORMAT'''
pv = EA.create_pivot_monthly('monthly')
pv.to_csv('Master_EA_Pivot.csv',index=False)

In [17]:
'''Create XLSX file which has shortened tabular data and a meta data file'''
short, meta = EA.create_xlsx_model_file()
short.to_csv('MASTER_EA_short.csv',index=False)
meta.to_csv('MASTER_EA_META.csv',index=False)

In [18]:
'''See what products you have a license for'''
license = EA.license
license.to_csv('EA_LICENSE.csv',index=False)

In [19]:
fd

,delivery_date,values,id,aspect,aspect_subtype,category,category_subtype,country,country_iso,description,forecast_start_date,frequency,lifecycle_stage,region,release_date,source,sub_region,unit
0,2015-01-01,969.78,49,demand,,oil_products,gasoline,United_Kingdom,GB,Total UK gasoline consumption,,monthly,actual,EUR,2023-01-03T18:38:00,"BEIS, Energy Aspects",,kt
1,2015-02-01,1018.05,49,demand,,oil_products,gasoline,United_Kingdom,GB,Total UK gasoline consumption,,monthly,actual,EUR,2023-01-03T18:38:00,"BEIS, Energy Aspects",,kt
2,2015-03-01,905.42,49,demand,,oil_products,gasoline,United_Kingdom,GB,Total UK gasoline consumption,,monthly,actual,EUR,2023-01-03T18:38:00,"BEIS, Energy Aspects",,kt
3,2015-04-01,1014.40,49,demand,,oil_products,gasoline,United_Kingdom,GB,Total UK gasoline consumption,,monthly,actual,EUR,2023-01-03T18:38:00,"BEIS, Energy Aspects",,kt
4,2015-05-01,1031.95,49,demand,,oil_products,gasoline,United_Kingdom,GB,Total UK gasoline consumption,,monthly,actual,EUR,2023-01-03T18:38:00,"BEIS, Energy Aspects",,kt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233786,2023-02-11,355.00,10816,generation,,power,,Canada,CA,Daily power generation data at Syncrude upgrad...,,daily,actual,NA,2023-02-15T11:28:00,"AER, Energy Aspects",,MW
233787,2023-02-12,359.00,10816,generation,,power,,Canada,CA,Daily power generation data at Syncrude upgrad...,,daily,actual,NA,2023-02-15T11:28:00,"AER, Energy Aspects",,MW
233788,2023-02-13,353.00,10816,generation,,power,,Canada,CA,Daily power generation data at Syncrude upgrad...,,daily,actual,NA,2023-02-15T11:28:00,"AER, Energy Aspects",,MW
233789,2023-02-14,365.00,10816,generation,,power,,Canada,CA,Daily power generation data at Syncrude upgrad...,,daily,actual,NA,2023-02-15T11:28:00,"AER, Energy Aspects",,MW


In [22]:
pv

,aspect,aspect_subtype,category,category_subtype,country_iso,description,forecast_start_date,frequency,lifecycle_stage,region,...,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01
182,production,,crude_oil,,DZ,"Algeria OPEC crude production, EA forecast, kb/d",2023-02-01,monthly,forecast,AFR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,production,,crude_oil,,AO,"Angola OPEC crude production, EA forecast, kb/d",2023-02-01,monthly,forecast,AFR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
166,production,,crude_oil,,CG,"Congo OPEC crude production, EA forecast, kb/d",2023-02-01,monthly,forecast,AFR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122,production,,crude_oil,,GQ,"Equatorial Guinea OPEC crude production, EA fo...",2023-02-01,monthly,forecast,AFR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,production,,crude_oil,,GA,"Gabon OPEC crude production, EA forecast, kb/d",2023-02-01,monthly,forecast,AFR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862,production,,crude_oil,,US,Monthly US crude oil production in Others in k...,2022-12-01,monthly,forecast,NA,...,1127.35,1124.31,1121.33,1118.41,1115.51,1112.6,1109.7,1106.83,1103.98,1101.18
2826,production,,crude_oil,,CA,Western Canada nonupgraded bitumen production ...,,monthly,forecast,NA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2798,production,,crude_oil,,CA,Western Canada synthetic crude production in k...,,monthly,forecast,NA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2837,exports,,crude_oil,,,NWE balance - total exports from NWE countries...,2022-11-01,monthly,forecast,EUR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
license

,description,source,date_from_min,date_to_max,aspect,aspect_subtype,category,category_subtype,frequency,lifecycle_stage,region,unit,country,country_iso,sub_region,licensed,id
0,"Algeria OPEC crude production, EA forecast, kb/d",Energy Aspects,2010-01-01,2023-12-01,production,None,crude_oil,None,monthly,forecast,AFR,kbbl_d,Algeria,DZ,None,yes,1
1,"Angola OPEC crude production, EA forecast, kb/d",Energy Aspects,2010-01-01,2023-12-01,production,None,crude_oil,None,monthly,forecast,AFR,kbbl_d,Angola,AO,None,yes,2
2,"Congo OPEC crude production, EA forecast, kb/d",Energy Aspects,2010-01-01,2023-12-01,production,None,crude_oil,None,monthly,forecast,AFR,kbbl_d,Congo,CG,None,yes,3
3,"Equatorial Guinea OPEC crude production, EA fo...",Energy Aspects,2010-01-01,2023-12-01,production,None,crude_oil,None,monthly,forecast,AFR,kbbl_d,Equatorial_Guinea,GQ,None,yes,4
4,"Gabon OPEC crude production, EA forecast, kb/d",Energy Aspects,2010-01-01,2023-12-01,production,None,crude_oil,None,monthly,forecast,AFR,kbbl_d,Gabon,GA,None,yes,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9946,Monthly Albania LNG imports in Mt,"GTT, Energy Aspects",2010-01-01,2025-12-01,imports,None,natural_gas,LNG,monthly,forecast,EUR,Mt,Albania,AL,None,no,15600
9947,Monthly Cyprus LNG imports in Mt,"GTT, Energy Aspects",2010-01-01,2025-12-01,imports,None,natural_gas,LNG,monthly,forecast,EUR,Mt,Cyprus,CY,None,no,15601
9948,Monthly Malta LNG imports in Mt,"GTT, Energy Aspects",2010-01-01,2025-12-01,imports,None,natural_gas,LNG,monthly,forecast,EUR,Mt,Malta,MT,None,no,15602
9949,Total World LNG exports including adjustment f...,"GTT, Energy Aspects",2010-01-01,2025-12-01,exports,None,natural_gas,LNG,monthly,forecast,WORLD,Mt,None,None,None,no,15666
